<a href="https://colab.research.google.com/github/dineshdaultani/CNN_training/blob/master/notebooks/2020_07_25/MNIST_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a tutorial for how to train a CNN model on MNIST dataset. MNIST contains pictures of handwritten digits. 

References: 
Code: https://keras.io/examples/mnist_cnn/
MNIST: http://yann.lecun.com/exdb/mnist/

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [0]:
batch_size = 128
num_classes = 10
epochs = 5
# input image dimensions
img_rows, img_cols = 28, 28

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.2652 - accuracy: 0.9191 - val_loss: 0.0539 - val_accuracy: 0.9825
Epoch 2/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.0859 - accuracy: 0.9748 - val_loss: 0.0424 - val_accuracy: 0.9860
Epoch 3/5
60000/60000 [==============================] - 9s 144us/step - loss: 0.0652 - accuracy: 0.9808 - val_loss: 0.0335 - val_accuracy: 0.9882
Epoch 4/5
60000/60000 [==============================] - 9s 143us/step - loss: 0.0522 - accuracy: 0.9847 - val_loss: 0.0331 - val_accuracy: 0.9889
Epoch 5/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.0461 - accuracy: 0.9862 - val_loss: 0.0293 - val_accuracy: 0.9905


In [20]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02929231648730056
Test accuracy: 0.9904999732971191
